In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Summarization_Task


/content/drive/My Drive/Colab Notebooks/Text_Summarization_Task


# **Extractive Model #1 Simple Model**
I started by making a simple extractive text summarization.  

It is a really simple model where you just give scores to words and sentences based on how frequent they are without having meaning behind words by not using any language models.

This can be easily applied to any language but it gives a rather dull results due to the lack of understanding to the language itself as it is basically a language agnostic approach.

The Goal of this model is not the best or even good results but rather building the foundation and the sitting stone in the project where you I can improve from this step.e

## **Running preprocessing Notebook to use it's functions**

A noteBook I made that contains the 2 functions used in most Model.  
It contains preprocess function for data preprocessing and read_wiki to read files from wikipedia and getting them ready in a string.

For more information about the two function make sure the read the notebook preprocessing.ipynb

In [0]:
%run Preprocessing.ipynb


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Getting The Data**

I try to work on a wikipedia page as it is considred a good article about a certain topic that would be nice to summarize.

I will use the read_wiki function from the preprocessing notebook to read and remove html tags.

Then I will use the preprocess function to clean the data.

you can read about those functions in more details from notebook preprocessing

I am going to read an English and Arabic articles


In [0]:
English_article = read_wiki('https://en.wikipedia.org/wiki/20th_century')
Arabic_article = read_wiki('https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B1%D8%A8_%D8%A7%D9%84%D8%B9%D8%A7%D9%84%D9%85%D9%8A%D8%A9_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A%D8%A9')

In [0]:
English_article

'The 20th (twentieth) century was a century that began on\nJanuary 1, 1901[1] and ended on December 31, 2000.[2] It was the tenth and final century of the 2nd millennium.\nThe 20th century was dominated by a chain of events that heralded significant changes in world history as to redefine the era: flu pandemic, World War I and World War II, nuclear power and space exploration, nationalism and decolonization, the Cold War and post-Cold War conflicts; intergovernmental organizations and cultural homogenization through developments in emerging transportation and communications technology; poverty reduction and world population growth, awareness of environmental degradation, ecological extinction;[3][4] and the birth of the Digital Revolution, enabled by the wide adoption of MOS transistors and integrated circuits. It saw great advances in power generation, communication and medical technology that by the late 1980s allowed for near-instantaneous worldwide computer communication and geneti

## **Data Preproccessing**

Furthermore, we’ll create a dictionary table having the frequency of occurrence of each of the words in the text but while doing so we will tokenize and clean the data

We clean the data by doing just the following:

*   Remove stopwords
*   Stemming using PortStemmer


In [0]:
from nltk.tokenize import word_tokenize, sent_tokenize

def _create_dictionary_table(text_string , l ):
  clean_article = preprocess(text_string , lang = l , rm_extras= False , lower = False , rm_tashkel= False , rm_contraction= False )
  words = word_tokenize(clean_article)
  frequency_table = dict()
  for wd in words:
      if wd in frequency_table:
          frequency_table[wd] += 1
      else:
          frequency_table[wd] = 1

  return frequency_table

## **Finding the weighted frequencies of the sentences**

To evaluate the score for every sentence in the text, we’ll be analyzing the frequency of occurrence of each term. In this case, we’ll be scoring each sentence by its words; that is, adding the frequency of each important word found in the sentence. 



In [0]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

       

    return sentence_weight

Importantly, to ensure long sentences do not have unnecessarily high scores over short sentences, we divided each score of a sentence by the number of words found in that sentence.

Also, to optimize the dictionary’s memory, we arbitrarily added sentence[:7], which refers to the first 7 characters in each sentence. However, for longer documents, where you are likely to encounter sentences with the same first n_chars, it’s better to use hash functions or smart index functions to take into account such edge-cases and avoid collisions. 

## **Calculating the threshold of the sentences**

To further tweak the kind of sentences eligible for summarization, we’ll create the average score for the sentences. With this threshold, we can avoid selecting the sentences with a lower score than the average score. 

In [0]:
def _calculate_average_score(sentence_weight) -> int:
   
    #calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    #getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score


## **Getting the summary**

In [0]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary


## **Wrapping Up**



This functions run all the previous functions in that order:


*   Frist it create a frequency table
*   Then tokenizing the sentences
*   Create the algorithm for scoring sentences that was describe above
*   Calculate sentence threshhold which is the average weight
*   Create the summary


In [0]:
def _run_article_summary(article , l):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article , l)

    #tokenizing the sentences
    sentences = sent_tokenize(article)
    #print(frequency_table)
    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)

    return article_summary

## **Results**

### **English Article**

In [0]:
En_summary_results = _run_article_summary(English_article , 'en')
print(En_summary_results)

 [2] It was the tenth and final century of the 2nd millennium. [12]
The century had the first global-scale total wars between world powers across continents and oceans in World War I and World War II. The victorious Bolsheviks then established the Soviet Union, the world's first communist state. In total, World War II left some 60 million people dead. During the century, the social taboo of sexism fell. By the end of the 20th century, women had the same legal rights as men in many parts of the world, and racism had come to be seen as abhorrent. The people of the Indian subcontinent, a sixth of the world population at the end of the 20th century, had attained an indigenous independence for the first time in centuries. Disease threatened to destabilize many regions of the world. World population increased from about 1.6 billion people in 1901 to 6.1 billion at the century's end. Many new genres of music were established during the 20th century. Film as an artistic medium was created in t

### **Arabic Article**

In [0]:
Ar_summary_results = _run_article_summary(Arabic_article , 'ar')
print(Ar_summary_results)

 إنشاء الأمم المتحدة. [6]
كانت الإمبراطورية اليابانية قد أعلنت الحرب على جمهورية الصين في 7 يوليو 1937، حيث هدفت إلى السيطرة على آسيا والمحيط الهادي،[7] إلا أن البداية الفعلية للحرب تعتبر في الأول من سبتمبر عام 1939،[8] وذلك عندما قامت ألمانيا باجتياح بولندا، وتوالت بعدها إعلانات الحرب على ألمانيا من قبل فرنسا والمملكة المتحدة. بقيت المعركة الأساسية في الحرب هي بين دول المحور من جهة، والمملكة المتحدة إلى جانب دول الكومنولث من جهة أخرى، إضافة إلى حملة في شمال إفريقيا وحملة أخرى في شرق أفريقيا، إضافة إلى معركة برلين الجوية وقصف لندن، وحملة البلقان، ومعركة المحيط الأطلسي. انتهت الحرب في أوروبا بغزو الحلفاء لألمانيا، وسيطرة الاتحاد السوفييتي على برلين والاستسلام غير المشروط من قبل ألمانيا في 8 مايو عام 1945. رغم أن حركة السلامية ازدادت قوة بعد الحرب العالمية الأولى،[25] كان من نتائج الحرب العالمية الأولى أن زادت روح القومية، والرغبة في الانتقام في دول أوروبية عديدة، وكان ذلك واضحًا خصوصًا في ألمانيا بعد خسارتها لجزء ملحوظ من أراضيها ومستعمراتها وأموالها بسبب معاهدة فرساي، حيث خسرت ألمانيا 

We Can see it gives us a extracted sentences of the highest score based on the used algorithm. It is not the best results but still something we can start with.